<a href="https://colab.research.google.com/github/ArshHp/LLM/blob/main/RAG_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

End to End RAG Pipeline using LLAMA Index & Langchain Framework

In [1]:
import nest_asyncio
nest_asyncio.apply()

import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install -q python-dotenv llama-index sentence-transformers
!pip install -q llama-index-embeddings-huggingface
!pip install -q langchain-groq
!pip install -q llama-index-llms-langchain
!pip install -q langchain langchain_community
!pip install -q google-generativeai
!pip install -q llama-index.llms.gemini --upgrade --no-cache-dir

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265

In [3]:
#Load the Env file
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

Load the Dataset
TODO: Provide the browse option to load the files

In [4]:
LLM_MODEL="models/gemini-2.0-flash"
EMBEDDING_MODEL="sentence-transformers/all-MiniLM-L6-v2"
DEVICE="cpu" #TODO:
#Add the code for if cuda exist else configue device as cpu, no need to setup torch now
#if torch.cuda.is_available():
 #   DEVICE = "cuda"


In [6]:
from llama_index.llms.gemini import Gemini
import os

# Configure Gemini LLM
llm = Gemini(model= LLM_MODEL, api_key=os.getenv("GOOGLE_API_KEY"))

# Define the Hugging Face embedding model (assuming embed_model was defined previously)
# If not, define it here:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
embed_model = HuggingFaceEmbedding(
model_name=EMBEDDING_MODEL,
device=DEVICE  # or "cuda"
 )

# Configure Settings to use the defined LLM and Embedding Model
Settings.llm = llm
Settings.embed_model = embed_model # Use the embed_model defined earlier

<ipython-input-6-fca7a1567b33>:5: DeprecationWarning: Call to deprecated class Gemini. (Should use `llama-index-llms-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/llm/google_genai/)
  llm = Gemini(model= LLM_MODEL, api_key=os.getenv("GOOGLE_API_KEY"))


In [39]:
from llama_index.core import SimpleDirectoryReader

#To load the set of documents from a directory
documents = SimpleDirectoryReader('/content/sample_data/data/').load_data()

In [40]:
print (documents[2])

Doc ID: a3e8c883-dd7f-43da-8e70-1a1083c5fa8f
Text: Figure 1: The Transformer - model architecture. The Transformer
follows this overall architecture using stacked self-attention and
point-wise, fully connected layers for both the encoder and decoder,
shown in the left and right halves of Figure 1, respectively. 3.1
Encoder and Decoder Stacks Encoder: The encoder is composed of a stack
of N = 6 i...


Chunking using Sentence Window Node Parser

In [41]:
from llama_index.core.node_parser import SentenceWindowNodeParser
sentence_window_parser = SentenceWindowNodeParser(
     # how many sentences on either side to capture
    window_size=3,

    # the metadata key that holds the window of surrounding sentences
    window_metadata_key="window",

    # the metadata key that holds the original sentence
    original_text_metadata_key="original_sentence",
)
nodes = sentence_window_parser.get_nodes_from_documents(documents)
print ("Total length of splitted nodes: ", len(nodes))

Total length of splitted nodes:  720


In [42]:
#Lets look at the chunks
nodes[0].to_dict()

{'id_': '473feefc-8927-4b43-a5b1-209c20963b56',
 'embedding': None,
 'metadata': {'page_label': '1',
  'file_name': 'Attention_Paper.pdf',
  'file_path': '/content/sample_data/data/Attention_Paper.pdf',
  'file_type': 'application/pdf',
  'file_size': 2215244,
  'creation_date': '2025-05-27',
  'last_modified_date': '2025-05-27',
  'window': 'Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\n Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence tra

In [43]:
#!pip install --upgrade --force-reinstall -q llama-index llama-index-vector-stores-chroma llama-index-embeddings-huggingface llama-index-llms-langchain langchain-groq chromadb
#!pip install --upgrade --force-reinstall -q llama-index.llms.gemini --upgrade --no-cache-dir


In [44]:
"""
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext
import chromadb

#Setup ChromaDB persistent client
chroma_client = chromadb.PersistentClient(path="./chroma_db")  # persists to disk
chroma_collection = chroma_client.get_or_create_collection("rag_index")

#Setup Chroma Vector Store
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# Setup storage context and build the index
storage_context = StorageContext.from_defaults(vector_store=vector_store)
"""

'\nfrom llama_index.vector_stores.chroma import ChromaVectorStore\nfrom llama_index.storage.storage_context import StorageContext\nimport chromadb\n\n#Setup ChromaDB persistent client\nchroma_client = chromadb.PersistentClient(path="./chroma_db")  # persists to disk\nchroma_collection = chroma_client.get_or_create_collection("rag_index")\n\n#Setup Chroma Vector Store\nvector_store = ChromaVectorStore(chroma_collection=chroma_collection)\n\n# Setup storage context and build the index\nstorage_context = StorageContext.from_defaults(vector_store=vector_store)\n'

In [45]:
from llama_index.core import VectorStoreIndex
from llama_index.core.postprocessor import MetadataReplacementPostProcessor

#Build the indexing using the configured Settings
#sentence_index = VectorStoreIndex(nodes,storage_context=storage_context,show_progress=True) # Settings are automatically picked up
sentence_index = VectorStoreIndex(nodes,show_progress=True) # Settings are automatically picked up
#sentence_index.storage_context.persist(persist_dir="./storage")

query_engine = sentence_index.as_query_engine(
    similarity_top_k=2,
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ]
)


Generating embeddings:   0%|          | 0/720 [00:00<?, ?it/s]

In [ ]:
from llama_index import load_index_from_storage
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext
import chromadb

# Step 1: Connect to existing ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = chroma_client.get_or_create_collection("rag_index")

# Step 2: Load vector store and storage context
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(
    vector_store=vector_store,
    persist_dir="./storage"
)

# Step 3: Reload the index
index = load_index_from_storage(storage_context)
query_engine = index.as_query_engine(
    similarity_top_k=2,
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ]
)

In [46]:
from llama_index.core.response.notebook_utils import display_response
sentence_window_response = query_engine.query("List the authors of attention paper?")
print (sentence_window_response.response)


The authors mentioned in the provided text are: Rafal Jozefowicz, Oriol Vinyals, Mike Schuster, Noam Shazeer, Yonghui Wu, Łukasz Kaiser, Samy Bengio, Ofir Press, Lior Wolf, Rico Sennrich, Barry Haddow, Alexandra Birch, Azalia Mirhoseini, Krzysztof Maziarz, Andy Davis, Quoc Le, Geoffrey Hinton, and Jeff Dean.



In [47]:
#print total nodes retrived
print ("Total Nodes Retrived:",len(sentence_window_response.source_nodes) )
for node in sentence_window_response.source_nodes:
    print("\n\n---- Response Source Info ----")
    print("Retrived Text:", node.node.get_content())
    print("Metadata:", node.node.metadata)
    print("Nodes Score:", node.score)
    print("Additional Content:", node.node.metadata["window"])
    print("-------------------------------")


Total Nodes Retrived: 2


---- Response Source Info ----
Retrived Text: Self-training PCFG grammars with latent annotations
across languages.  In Proceedings of the 2009 Conference on Empirical Methods in Natural
Language Processing, pages 832–841.  ACL, August 2009.
 [15] Rafal Jozefowicz, Oriol Vinyals, Mike Schuster, Noam Shazeer, and Yonghui Wu.  Exploring
the limits of language modeling.  arXiv preprint arXiv:1602.02410, 2016.
 [16] Łukasz Kaiser and Samy Bengio. 
Metadata: {'page_label': '11', 'file_name': 'Attention_Paper.pdf', 'file_path': '/content/sample_data/data/Attention_Paper.pdf', 'file_type': 'application/pdf', 'file_size': 2215244, 'creation_date': '2025-05-27', 'last_modified_date': '2025-05-27', 'window': 'Self-training PCFG grammars with latent annotations\nacross languages.  In Proceedings of the 2009 Conference on Empirical Methods in Natural\nLanguage Processing, pages 832–841.  ACL, August 2009.\n [15] Rafal Jozefowicz, Oriol Vinyals, Mike Schuster, Noam Shazeer

In [48]:
sentence_window_response = query_engine.query("What is multi-head attention")
print (sentence_window_response.response)

It consists of several attention layers running in parallel. The Transformer uses it in three different ways: in "encoder-decoder attention" layers, in the encoder with self-attention layers.



In [49]:
sentence_window_response = query_engine.query("Provide an elboratove answer for What is multi-head attention")
print (sentence_window_response.response)

Multi-Head Attention involves multiple attention layers that operate in parallel. It is used to counteract the reduced effective resolution caused by averaging attention-weighted positions.



In [50]:
sentence_window_response = query_engine.query("What are the minimum installment amount for monthly, Half-Yearly, Quaterly and Yearly payments?")
print (sentence_window_response.response)

The minimum installment amounts are: Rs. 5000 for monthly, Rs. 15000 for quarterly, Rs. 25000 for half-yearly, and Rs. 50000 for yearly payments.



In [58]:
# Turn it into a chat engine
# Create a query engine
chat_engine  = sentence_index.as_chat_engine(
    chat_mode="condense_question",
    verbose=True
)

# Simulate chat
print("Ask a question (type 'exit' to quit):")
while True:
    user_input = input("User: ")
    if user_input.lower() == "exit":
        break
    response = chat_engine.chat(user_input)
    print("Bot:", response.response)



Ask a question (type 'exit' to quit):
User: What is Attention 
Querying with: What is Attention 
Bot: The document discusses attention distributions and expresses excitement about the future of attention-based models, with plans to apply them to other tasks.

User: Tell me more
Querying with: The document discusses attention distributions and expresses excitement about the future of attention-based models, with plans to apply them to other tasks. Can you tell me more about this?

Bot: The document includes a discussion and presentation of attention distribution examples in the appendix. There is also excitement expressed about the future of attention-based models, with intentions to use them on other tasks.

User: What is the main principal behind it
Querying with: Based on the document previously discussed, which mentions attention distributions and excitement about applying attention-based models to other tasks, what is the main principle behind attention?

Bot: Attention distributio

In [59]:
from fastapi import FastAPI, Request
from pydantic import BaseModel

app = FastAPI()

class ChatInput(BaseModel):
    message: str

@app.post("/chat")
async def chat_endpoint(input: ChatInput):
    response = chat_engine.chat(input.message)
    return {"response": response.response}
